In [1]:
! pip install cma

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import timeit

import numpy as np
import cma
from IPython.display import display, clear_output

from anguilla.optimizers.cma import CMA, StrategyParameters, StoppingConditions
from anguilla.fitness.benchmark import Sphere, SumSquares

In [4]:
def test_with_pycma(n=4):
    fitness_function = Sphere()
    initial_point = fitness_function.propose_initial_point(n)
    initial_sigma = 0.3
    cma.fmin(fitness_function, initial_point, initial_sigma, options={'CMA_Active': 0})

test_with_pycma()

(4_w,8)-CMA-ES (mu_w=2.6,w_1=52%) in dimension 4 (seed=556575, Thu Nov 26 12:19:19 2020)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1      8 3.077208987461802e+00 1.0e+00 2.87e-01  3e-01  3e-01 0:00.0
    2     16 1.972025418004691e+00 1.2e+00 2.87e-01  3e-01  3e-01 0:00.0
    3     24 8.563327312513337e-01 1.2e+00 3.54e-01  3e-01  4e-01 0:00.0
  100    800 8.904745969751205e-15 1.5e+00 2.91e-06  6e-08  8e-08 0:00.3
  107    856 1.517587209868988e-15 1.4e+00 1.43e-06  2e-08  3e-08 0:00.3
termination on tolfun=1e-11 (Thu Nov 26 12:19:24 2020)
final/bestever f-value = 7.400358e-16 3.733785e-16
incumbent solution: [-1.221413785898568e-08, 1.6757729576581114e-08, 8.571142920895e-09, -1.5380656831008103e-08]
std deviation: [2.573309680349744e-08, 2.823659464002505e-08, 2.3413816352184244e-08, 2.2437228132830215e-08]


In [5]:
def test_with_prototype(n=4):
    parameters = StrategyParameters(n=n)
    conditions = StoppingConditions(n=n,
                                    population_size=parameters.population_size,
                                    ftarget=1e-14,
                                    )
    fitness_function = Sphere()
    initial_point = fitness_function.propose_initial_point(n)
    optimizer = CMA(initial_point=initial_point,
                    initial_sigma=0.3,
                    strategy_parameters=parameters,
                    stopping_conditions=conditions)
    while not optimizer.stop():
        candidate_solutions = optimizer.ask()
        function_values = fitness_function(candidate_solutions.T)
        ranked_indices = optimizer.rank(candidate_solutions, function_values)
        optimizer.tell(candidate_solutions, ranked_indices)
        if (optimizer._fevals % 20) == 0:
            display(optimizer._fevals, optimizer._best_value, optimizer._best_solution)

test_with_prototype(n=4)

20

0.13964790258374316

array([-0.03554292, -0.18841616, -0.28480109, -0.14755438])

40

0.13964790258374316

...

1.2490559918109497e-14

array([-5.69125997e-08,  1.85515115e-08, -5.01520277e-08, -7.99508064e-08])

700

6.898520830577873e-15

array([ 1.71231329e-08,  1.79222549e-08, -7.26058950e-08,  3.18197414e-08])